In [2]:
import propka

In [ ]:
proka.run(
    pdb_file="7SL6_medoid_cluster_1_AA_renumbered_SSBOND_fix_heavy_re.pdb", 
    output_file="7SL6_medoid_cluster_1_AA_renumbered_SSBOND_fix_heavy_re.out",
    pH=7.4,
    forcefield="amber",
    ion_strength=0.1,
    temperature=298.15,
    n_threads=4,
    verbose=True)

NameError: name 'proka' is not defined

In [18]:
d = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

In [21]:
list(d.keys())

['CYS',
 'ASP',
 'SER',
 'GLN',
 'LYS',
 'ILE',
 'PRO',
 'THR',
 'PHE',
 'ASN',
 'GLY',
 'HIS',
 'LEU',
 'ARG',
 'TRP',
 'ALA',
 'VAL',
 'GLU',
 'TYR',
 'MET']

In [18]:
def auto_detect_types(water_model='OPC'):
    #input4amber.pdb
    # source leaprc.protein.ff19SB
    # source leaprc.water.opc
    # source leaprc.lipid21
    #sed -i '1s/^/task goes here\n/' todo.txt

    forcefields = {'protein': 'protein.ff19SB', 'lipid': 'lipid21', 'dna': 'DNA.OL24', 'rna': 'RNA.OL3', 'water': {'OPC': 'water.opc', 'TIP3P': 'water.tip3p', 'TIP4PEW': 'water.tip4pew'}}

    protein = ['CYS','ASP','SER','GLN','LYS','ILE','PRO','THR','PHE','ASN','GLY','HIS','LEU','ARG','TRP','ALA','VAL','GLU','TYR','MET']
    lipid = ['PC', 'PA', 'OL', 'CHL']
    nucleic = ['A','C','G']
    rna = ['U']
    dna = ['T']
    with open('../tests/input4amber.pdb', 'r') as f:
        lines = f.readlines()

    resnames = []
    
    ff_types = []

    for line in lines:

        l = line.split()

        if l[0] == 'END':
            break

        if l[0] in ['ATOM', 'HETATM']:
            resnames.append(l[3])

    resnames = list(set(resnames))



    for resname in resnames:
        if resname in protein:
            if 'protein' not in ff_types:
                ff_types.append('protein')
        elif resname in lipid:
            if 'lipid' not in ff_types:
                ff_types.append('lipid')
        # elif resname in nucleic:
        #     if 'nucleic' not in ff_types:
        #         ff_types.append('nucleic')
        elif resname in rna:
            if 'rna' not in ff_types:
                ff_types.append('rna')
        elif resname in dna:
            if 'dna' not in ff_types:
                ff_types.append('dna')
        else:
            print('type not found', resname, ff_types)

    leaprc = ''

    for ff_type in sorted(ff_types, key=lambda x: len(x), reverse=True):
        leaprc += f'source leaprc.{forcefields[ff_type]}\\n'

    leaprc += f'source leaprc.{forcefields['water'][water_model.upper()]}\\n'
            
    return leaprc

In [20]:
leap = auto_detect_types(water_model='tip4pew')
print(leap)

type not found A ['protein', 'rna']
type not found C ['protein', 'rna']
type not found G ['protein', 'rna']
type not found LIG ['protein', 'rna']
source leaprc.protein.ff19SB\nsource leaprc.RNA.OL3\nsource leaprc.water.tip4pew\n


In [4]:
forcefields = {'protein': 'protein.ff19SB', 'lipid': 'lipid21', 'dna': 'DNA.OL24', 'rna': 'RNA.OL3'}


In [9]:
sorted(list(forcefields.keys()), key=lambda x: len(x), reverse=True)

['protein', 'lipid', 'dna', 'rna']

In [ ]:

run(f'gmx insert-molecules -f tests/input4amber.pdb -ci models/{args.water.lower()}.gro -o tests/input4amber.pdb -nmol 1')

NameError: name 'water_model' is not defined

In [21]:
import argparse
import subprocess
from pathlib import Path
import sys
import os
import shutil
import yaml

def run(cmd, shell=True, check=True):
    if isinstance(cmd, list):
        print(f"Running: {' '.join(str(x) for x in cmd)}")
    else:
        print(f"Running: {cmd}")
    subprocess.run(cmd, shell=shell, check=check)

In [55]:
run(f"sed -i '1s/^/{leap}/' tleap.in")

Running: sed -i '1s/^/source leaprc.protein.ff19SB\nsource leaprc.RNA.OL3\n/' tleap.in


In [20]:
with open('coby_input.yaml') as f:
    coby_args = yaml.safe_load(f)

In [21]:
coby_cmd = ["python3", "-m", "COBY"] + coby_args

In [22]:
s = ' '.join(str(x) for x in coby_cmd)

In [23]:
s

'python3 -m COBY -box 18 18 23 -membrane lipid:POPC:6:apl:.69:params:Amber lipid:CHL:4:apl:.41:params:Amber optimize_run:False grid_maker_algorithm:no_groups center:0:0:-7 -protein file:{protein_pdb} charge:-28 cen_method:875-915:1790-1829 cz:4 -solvation solv:OPC:1:params:AA_Amber pos:Na+:1:params:AA neg:Cl-:1:params:AA count:True bead_radius:0.12 kick:0.03 buffer:0.1 protein_extra_buffer:0.0 -itp_input file:coby_input.top -molecule_import file:toppar/POPC_rotated.pdb moleculetype:POPC params:Amber -molecule_import file:toppar/CHL_rotated.pdb moleculetype:CHL params:Amber -out_sys COBY.pdb -out_top COBY.top'